In [ ]:
"""
This notebook works through the logic for pre & post cooling
"""

In [147]:
import collections

In [148]:
class env(object):
    def __init__(self):
        self.memory = 3
        self.capacity = 5
        self.reset()

    def reset(self):    
        self.precool_hist = collections.deque(maxlen=self.memory)
        self.postcool_hist = collections.deque(maxlen=self.memory)
        print('reset')
        return None

    def update_hists(self, precool, postcool):
            self.precool_hist.append(precool)
            self.postcool_hist.append(postcool)
            return None    
        
    def step(self,action):
        
        """
        this function steps through the environment
        """
    
        precool_hist = sum(self.precool_hist)
        postcool_hist = sum(self.postcool_hist)

        #  should we start a precooling event
        if precool_hist == 0 and action == 1 and postcool_hist == 0:
            print('starting pre-cooling event')
            self.update_hists(1, 0)

        #  are we already in a precooling event
        elif precool_hist > 0 and precool_hist < self.memory and self.precool_hist[-1] == 1:
            print('continuing pre-cooling event')
            self.update_hists(1, 0)

        #  are we at the end of a precooling event
        #  aka should we start a postcooling event
        elif precool_hist == self.memory:
            print('ending pre-cooling event')
            print('starting post-cooling event')
            self.update_hists(0, 1)

        #  are we already in a postcooling event   
        elif postcool_hist > 0 and postcool_hist < self.memory and self.postcool_hist[-1] == 1:    
            print('continuing post-cooling event')
            self.update_hists(0, 1)

        #  are we ending a postcooling event
        elif postcool_hist == self.memory and precool_hist == 0:
            print('ending post-cooling event')
            self.update_hists(0, 0)

        else:
            print('nothing is happening')
            self.update_hists(0, 0)
            
        print('pre-cooling history {}'.format(self.precool_hist))
        print('post-cooling history {}'.format(self.postcool_hist)) 
        
        #  now calculating the effect on demand
        demand_adjustment = 0
        
        if self.precool_hist[-1] == 1:
            demand_adjustment = self.capacity
            
        if self.postcool_hist[-1] == 1:
            demand_adjustment = -self.capacity    
            
        print('demand adjustment is {}'.format(demand_adjustment))    
            
        return None    

In [149]:
e = env()

reset


In [150]:
e.step(1)

starting pre-cooling event
pre-cooling history deque([1], maxlen=3)
post-cooling history deque([0], maxlen=3)
demand adjustment is 5


In [151]:
e.step(1)

continuing pre-cooling event
pre-cooling history deque([1, 1], maxlen=3)
post-cooling history deque([0, 0], maxlen=3)
demand adjustment is 5


In [152]:
e.step(1)

continuing pre-cooling event
pre-cooling history deque([1, 1, 1], maxlen=3)
post-cooling history deque([0, 0, 0], maxlen=3)
demand adjustment is 5


In [153]:
e.step(1)

ending pre-cooling event
starting post-cooling event
pre-cooling history deque([1, 1, 0], maxlen=3)
post-cooling history deque([0, 0, 1], maxlen=3)
demand adjustment is -5


In [154]:
e.step(1)

continuing post-cooling event
pre-cooling history deque([1, 0, 0], maxlen=3)
post-cooling history deque([0, 1, 1], maxlen=3)
demand adjustment is -5


In [155]:
e.step(1)

continuing post-cooling event
pre-cooling history deque([0, 0, 0], maxlen=3)
post-cooling history deque([1, 1, 1], maxlen=3)
demand adjustment is -5


In [156]:
e.step(1)

ending post-cooling event
pre-cooling history deque([0, 0, 0], maxlen=3)
post-cooling history deque([1, 1, 0], maxlen=3)
demand adjustment is 0


In [157]:
e.step(1)

nothing is happening
pre-cooling history deque([0, 0, 0], maxlen=3)
post-cooling history deque([1, 0, 0], maxlen=3)
demand adjustment is 0


In [158]:
e.step(1)

nothing is happening
pre-cooling history deque([0, 0, 0], maxlen=3)
post-cooling history deque([0, 0, 0], maxlen=3)
demand adjustment is 0


In [159]:
e.step(1)

starting pre-cooling event
pre-cooling history deque([0, 0, 1], maxlen=3)
post-cooling history deque([0, 0, 0], maxlen=3)
demand adjustment is 5


In [160]:
e.step(1)

continuing pre-cooling event
pre-cooling history deque([0, 1, 1], maxlen=3)
post-cooling history deque([0, 0, 0], maxlen=3)
demand adjustment is 5


In [161]:
e.step(1)

continuing pre-cooling event
pre-cooling history deque([1, 1, 1], maxlen=3)
post-cooling history deque([0, 0, 0], maxlen=3)
demand adjustment is 5


In [162]:
e.step(1)

ending pre-cooling event
starting post-cooling event
pre-cooling history deque([1, 1, 0], maxlen=3)
post-cooling history deque([0, 0, 1], maxlen=3)
demand adjustment is -5


In [163]:
e.step(1)

continuing post-cooling event
pre-cooling history deque([1, 0, 0], maxlen=3)
post-cooling history deque([0, 1, 1], maxlen=3)
demand adjustment is -5


In [164]:
e.step(1)

continuing post-cooling event
pre-cooling history deque([0, 0, 0], maxlen=3)
post-cooling history deque([1, 1, 1], maxlen=3)
demand adjustment is -5


In [165]:
e.step(1)

ending post-cooling event
pre-cooling history deque([0, 0, 0], maxlen=3)
post-cooling history deque([1, 1, 0], maxlen=3)
demand adjustment is 0


In [166]:
e.step(0)

nothing is happening
pre-cooling history deque([0, 0, 0], maxlen=3)
post-cooling history deque([1, 0, 0], maxlen=3)
demand adjustment is 0


In [167]:
e.step(0)

nothing is happening
pre-cooling history deque([0, 0, 0], maxlen=3)
post-cooling history deque([0, 0, 0], maxlen=3)
demand adjustment is 0


In [168]:
e.step(1)

starting pre-cooling event
pre-cooling history deque([0, 0, 1], maxlen=3)
post-cooling history deque([0, 0, 0], maxlen=3)
demand adjustment is 5


In [169]:
e.step(0)

continuing pre-cooling event
pre-cooling history deque([0, 1, 1], maxlen=3)
post-cooling history deque([0, 0, 0], maxlen=3)
demand adjustment is 5
